In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import pandas as pd
from pprint import pprint

In [2]:
mars_news_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [3]:
news_response = requests.get(mars_news_url)

In [4]:
news_soup = bs(news_response.text, 'html.parser') 

In [21]:
news_results = news_soup.find_all('div', class_="slide") 

In [31]:
# news_results_first = news_soup.find('div', class_="slide") 
# news_title = news_results_first.find('div', class_="content_title").text
# top_title = news_title.replace("\n","")
# top_title_p = news_results_first.a.div.text.replace("\n","")

In [8]:
for result in news_results:
    try:
        news_title = result.find('div', class_="content_title").text
        news_p = result.a.div.text

        if (news_title and news_p):
            print(f'-------------{news_title}\n{news_p}')
    except AttributeError as e:
        print(e)

-------------

NASA's Curiosity Mars Rover Finds a Clay Cache




The rover recently drilled two samples, and both showed the highest levels of clay ever found during the mission.





-------------

Why This Martian Full Moon Looks Like Candy




For the first time, NASA's Mars Odyssey orbiter has caught the Martian moon Phobos during a full moon phase. Each color in this new image represents a temperature range detected by Odyssey's infrared camera.





-------------

NASA Garners 7 Webby Award Nominations




Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.





-------------

NASA's Opportunity Rover Mission on Mars Comes to End




NASA's Opportunity Mars rover mission is complete after 15 years on Mars. Opportunity's record-breaking exploration laid the groundwork for future missions to the Red Planet.





-------------

NASA's InSight Places First Instrument on Ma

In [ ]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False) 

In [ ]:
image_url= "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url) 

In [ ]:
image_html = browser.html
image_soup = bs(image_html, 'html.parser')

featured_image = image_soup.find('article', class_='carousel_item')["style"]

link_featured_image = featured_image.split("'")[1]

base_url = "https://www.jpl.nasa.gov"

featured_image_url = base_url + link_featured_image
featured_image_url

In [ ]:
weather_url = "https://twitter.com/MarsWxReport"
weather_response = requests.get(weather_url)
weather_soup = bs(weather_response.text, 'html.parser') 
print(weather_soup.prettify()) 

In [ ]:
mars_weather = weather_soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
mars_weather

In [ ]:
facts_url = "https://space-facts.com/mars/"
facts_response = requests.get(facts_url)
facts_soup = bs(facts_response.text, 'html.parser') 
print(facts_soup.prettify()) 

In [ ]:
fact_table= pd.read_html(facts_url)[1]
fact_table

In [ ]:
facts = fact_table.to_html(header=False, index=False, index_names=False)
pprint(facts)

In [ ]:
hemisphere_url= "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemisphere_url) 

In [ ]:
hemisphere_html = browser.html
hemisphere_soup = bs(hemisphere_html, 'html.parser')

new_image = hemisphere_soup.find_all("a", class_="itemLink product-item")

hemisphere_base_url = "https://astrogeology.usgs.gov"

new_image

In [ ]:
image_sources = []
image_url_list = []
titles = []
iteration = 0

for links in new_image:
    end_url = links.get("href")
    if end_url not in image_sources:
        image_sources.append(end_url)
        iteration += 1
        click_url = hemisphere_base_url + end_url
        browser.visit(click_url)
        titles.append(browser.find_by_tag('h2').text)        
        browser.find_link_by_text("Sample").first.click()
        print(browser.find_link_by_text("Sample"))
        print(browser.title)
        url_for_image = browser.windows[iteration].url
        image_url_list.append(url_for_image)

image_url_list

In [ ]:
pprint(titles)
pprint(image_url_list)

In [ ]:
hemisphere_image_urls = [
    {"title":titles[0], "img_url":image_url_list[0]},
    {"title":titles[1], "img_url":image_url_list[1]},
    {"title":titles[2], "img_url":image_url_list[2]},
    {"title":titles[3], "img_url":image_url_list[3]}
]

In [ ]:
hemisphere_image_urls

In [ ]:
i = 0
for picture in image_url_list:
    Picture_request = requests.get(picture)
    if Picture_request.status_code == 200:
        with open(f"{titles[i]}.jpg", 'wb') as new:
            new.write(Picture_request.content)
            i = i + 1